In [ ]:
# Importing the required libraries
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
# csv paths
maven = "Maven Airbnb reviews/Listingst.csv"
kaggle = "Kaggle NYC airbnb/AB_NYC_2019.csv"

# read csv
m_csv = pd.read_csv(maven, low_memory=False)
k_csv = pd.read_csv(kaggle)

In [ ]:
m_csv.head()

In [ ]:
k_csv.head()

In [ ]:
# Choose the needed columns for the dataframe
k_df = k_csv[[
    "id",
    "name",
    "host_name",
    "neighbourhood_group",
    "neighbourhood",
    "room_type",
    "price"
]]
m_df = m_csv[[
    "listing_id",
    "name", "host_since",
    "host_total_listings_count",
    "neighbourhood",
    "district",
    "city",
    "property_type",
    "accommodates",
    "review_scores_accuracy",
    "review_scores_cleanliness",
    "review_scores_location"
]]

In [ ]:
# Filter the maven sheet for New York data only
maven = m_df.loc[m_df["city"] == "New York"]

In [ ]:
# The maven filtered dataframe
maven.head()

In [ ]:
# The kaggle dataframe
k_df

In [ ]:
# The dataframes are: maven & k_df
## Checking unique values for: maven[district] & kaggle[neighbourhood_group]
maven["district"].unique()

In [ ]:
k_df["neighbourhood_group"].unique()

In [ ]:
## Checking unique values for: maven[neighbourhood] & kaggle[neighbourhood]
list_maven = []
maven_values = maven["neighbourhood"].unique()
for x in maven_values:
    list_maven.append(x)
    
list_kaggle = []
kaggle_values = k_df["neighbourhood"].unique()
for x in kaggle_values:
    list_kaggle.append(x)


In [ ]:
# checking for the outliers, to be removed from the kaggle dataframe
outliers = []
for x in list_kaggle:
    if not x in list_maven:
        outliers.append(x)
        
outliers

In [ ]:
# Deleting the outliers from kaggle so the datasets would match
kaggle = k_df.loc[
    (k_df['neighbourhood'] != "Bay Terrace, Staten Island")
    &
    (k_df['neighbourhood'] != "Breezy Point")
    &
    (k_df['neighbourhood'] != "Neponsit")
]

list_kaggle_new = []
kaggle_check = kaggle['neighbourhood'].unique()
for x in kaggle_check:
    list_kaggle_new.append(x)
    
list_kaggle_new.sort()
list_kaggle_new

In [ ]:
# Renaming columns in both dataframes
kaggle_final = kaggle.rename(columns = {
    "id" : "kid",
    "name" : "kaggle_house_description",
    "host_name" : "host_name",
    "neighbourhood_group" : "district",
    "neighbourhood" : "neighbourhood",
    "room_type" : "room_type",
    "price" : "price_$"
})

maven_final = maven.rename(columns = {
    "listing_id" : "mid",
    "name" : "maven_house_description",
    "host_since" : "first_on_market",
    "host_total_listings_count" : "listing_count",
    "neighbourhood" : "neighbourhood",
    "district" : "district",
    "city" : "city",
    "property_type" : "property_type",
    "accommodates" : "number_of_tenants_allowed",
    "review_scores_accuracy" : "total_review",
    "review_scores_cleanliness" : "cleanliness_review",
    "review_scores_location" : "location_review"
})

In [ ]:
kaggle_final.head()

In [ ]:
maven_final.head()

In [ ]:
# Connect to database
protocol = 'postgresql'
username = 'postgres'
password = 'Ruchi0619'
host = 'localhost'
port = 5432
database_name = 'Airbnb'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [ ]:
maven_final.to_sql(name='maven_final', con=engine, if_exists='append', index=False)
kaggle_final.to_sql(name='kaggle_final', con=engine, if_exists='append', index=False)

In [ ]:
pd.read_sql_query('select * from kaggle_final', con=engine).head()

In [ ]:
pd.read_sql_query('select * from kaggle_final', con=engine).head()